In [160]:
import pymongo
import pprint

In [2]:
conn = pymongo.MongoClient('mongodb:')
db = conn.get_database('documents')

예제 데이터 삽입

In [ ]:
db.things.insert_one({"x":1, "tags":["dog", "cat"]})
db.things.insert_one({"x":2, "tags":["cat"]})
db.things.insert_one({"x":3, "tags":["mouse", "cat", "dog"]})
db.things.insert_one({"x":4, "tags":[]})
db.things.insert_one({"x":5, "tags":["a", "b"]})

이제 map과 reduce 함수를 정의합니다. 이 경우 MongoDB Map/Reduce document에서와 동일한 작업을 수행합니다. 전체 컬렉션에서 태그 배열의 각 태그에 대한 발생 수를 count합니다.

map 함수는 배열의 각 태그에 대해 single(key, 1) 쌍을 내보냅니다.

In [ ]:
from bson.code import Code
map = Code("""
            function () {
                var self = this;
                self.tags.forEach(function(z) {
                    emit(z, 1);});
                }""" )

reduce 함수는 주어진 키에 대해 모든 내보낸 값에 대해 합계를 구합니다.

In [ ]:
reduce = Code("""
            function (key, values) {
                var total = 0;
                for (var i = 0; i < values.length; i++) {
                    total += values[i];}
                return total;
            } """)

map_reduce()함수를 호출하고 결과 collection을 출력합니다.

In [ ]:
result = db.things.map_reduce(map, reduce, out={merge:"things"})
for doc in result.find():
    print(doc)

In [ ]:
from bson.son import SON
db.things.map_reduce(map, reduce, out=SON([("replace", "results"), ("db", "outdb")]), full_response=True)

In [ ]:
reduce(lambda all_keys, rec_keys: all_keys | set(rec_keys), map(lambda d: d.keys(), db.things.find()), set() )

In [ ]:
table = []
cursor = db['things'].find({})
for document in cursor:
    table.extend(list(set(document.keys())))

In [ ]:
table

In [ ]:
import pprint
cursor = db.temp.aggregate([
  {"$project":{"o":{"$objectToArray":"$$ROOT"}}},
  {"$unwind":"$o"},
  {"$group":{"_id":0, "keys":{"$addToSet":"$o.k"}}}
])

for data in cursor:
    pprint.pprint(data)

In [189]:
import pprint
cursor = db['things'].aggregate([
  {
      "$project": # document pipeline을 reshape함
      {
          "o":
          {
              "$objectToArray":"$$ROOT"
          }
      }
  },
  {
      "$unwind":"$o" # array를 flat으로 만들어줌
  },
  {
      "$group":
      {
          "_id":0,
          "keys":
          {
              "$addToSet":"$o.k"
          }
      }
  }])

temp = list(cursor)
for data in cursor:
    pprint.pprint(data)

In [60]:
import pymongo

In [65]:
# 1개 업데이트
db['things'].update_one(
    {"item" : "jkl"},
    {"$set": {"price":0}}
)

In [69]:
# 다중 업데이트
db['things'].update_many(
    {"price" : {"$gte":5}}, # price가 5보다 같거나 크면
    {"$set" : {"quantity": "15"}}
)

In [72]:
db['things'].update_one(
    {"item":"jkl"},
    {"$unset": {"item":1}} # 필드 삭제
)

In [93]:
db['things'].update_one(
    {"item":"efg"},
    {"$set":{
        "favorite":{
            "cities" : ["Chicago", "Cheyenne"],
            "movies" : ["Casablanca", "For a Few Daller", "The Sting"]
        }
    }}
)

In [92]:
db['things'].update_one(
    {"favorite.cities":"Chicago"},
    # 배열에 2개 값이 있을 경우 하나로 변경됨
    {"$set":{"favorite.cities":"Chicago1"}}
)

In [123]:
db['things'].update_one(
    {"favorite.cities":"Chicago"},
     {"$addToSet":{"favorite.cities": "Chicago3"}}, # 배열에 없으면 추가함
    False, # upsert 옵션 : 찾는게 있으면 추가, 없으면 삽입
    True, # 다중 업데이트
)

In [ ]:
# 조건에 맞는 전체 업데이트
db['table'].update_many(
    {"start_date": None},
     {"$set":{"start_date": "123"}}
)

In [125]:
db.things.remove({}) # 컬렉션 제외 하고 도큐먼트만 삭제
db.things.drop() # 도큐먼트, 컬렉션 모두 삭제

인덱스

In [132]:
for i in range(100):
    db.things.insert_one({"num": i})

In [143]:
db.things.estimated_document_count()

100

In [146]:
cursor = db.things.find({"num": {"$gt":30, "$lt":40}})

for item in cursor:
    print(item)

{'_id': ObjectId('5ff2cabf30cd3ec39f73238c'), 'num': 31}
{'_id': ObjectId('5ff2cabf30cd3ec39f73238d'), 'num': 32}
{'_id': ObjectId('5ff2cabf30cd3ec39f73238e'), 'num': 33}
{'_id': ObjectId('5ff2cabf30cd3ec39f73238f'), 'num': 34}
{'_id': ObjectId('5ff2cabf30cd3ec39f732390'), 'num': 35}
{'_id': ObjectId('5ff2cabf30cd3ec39f732391'), 'num': 36}
{'_id': ObjectId('5ff2cabf30cd3ec39f732392'), 'num': 37}
{'_id': ObjectId('5ff2cabf30cd3ec39f732393'), 'num': 38}
{'_id': ObjectId('5ff2cabf30cd3ec39f732394'), 'num': 39}


데이터베이스가 쿼리를 받을 때 이를 실행하는 방법에 대해 계획을 세워야 하며 이를 
쿼리 플랜이라 부름. explain은 이 쿼리를 실행한 결과를 보여줌

In [174]:
db.things.find({"num": {"$gt":30, "$lt":40}}).explain()

{'queryPlanner': {'plannerVersion': 1,
  'namespace': 'documents.things',
  'indexFilterSet': False,
  'parsedQuery': {'$and': [{'num': {'$lt': 40}}, {'num': {'$gt': 30}}]},
  'winningPlan': {'stage': 'COLLSCAN',
   'filter': {'$and': [{'num': {'$lt': 40}}, {'num': {'$gt': 30}}]},
   'direction': 'forward'},
  'rejectedPlans': []},
 'executionStats': {'executionSuccess': True,
  'nReturned': 9,
  'executionTimeMillis': 0,
  'totalKeysExamined': 0,
  'totalDocsExamined': 100,
  'executionStages': {'stage': 'COLLSCAN',
   'filter': {'$and': [{'num': {'$lt': 40}}, {'num': {'$gt': 30}}]},
   'nReturned': 9,
   'executionTimeMillisEstimate': 0,
   'works': 102,
   'advanced': 9,
   'needTime': 92,
   'needYield': 0,
   'saveState': 0,
   'restoreState': 0,
   'isEOF': 1,
   'direction': 'forward',
   'docsExamined': 100},
  'allPlansExecution': []},
 'serverInfo': {'host': 'c9bbbb105683',
  'port': 27017,
  'version': '4.4.2',
  'gitVersion': '15e73dc5738d2278b688f8929aee605fe4279b0e'},
 'o

In [168]:
db.things.find({"num": {"$gt":80}}).explain()

{'queryPlanner': {'plannerVersion': 1,
  'namespace': 'documents.things',
  'indexFilterSet': False,
  'parsedQuery': {'num': {'$gt': 80}},
  'winningPlan': {'stage': 'COLLSCAN',
   'filter': {'num': {'$gt': 80}},
   'direction': 'forward'},
  'rejectedPlans': []},
 'executionStats': {'executionSuccess': True,
  'nReturned': 19,
  'executionTimeMillis': 0,
  'totalKeysExamined': 0,
  'totalDocsExamined': 100,
  'executionStages': {'stage': 'COLLSCAN',
   'filter': {'num': {'$gt': 80}},
   'nReturned': 19,
   'executionTimeMillisEstimate': 0,
   'works': 102,
   'advanced': 19,
   'needTime': 82,
   'needYield': 0,
   'saveState': 0,
   'restoreState': 0,
   'isEOF': 1,
   'direction': 'forward',
   'docsExamined': 100},
  'allPlansExecution': []},
 'serverInfo': {'host': 'c9bbbb105683',
  'port': 27017,
  'version': '4.4.2',
  'gitVersion': '15e73dc5738d2278b688f8929aee605fe4279b0e'},
 'ok': 1.0,
 '$clusterTime': {'clusterTime': Timestamp(1609810376, 1),
  'signature': {'hash': b'\x00\

In [175]:
db.things.create_index(
    [("num", 
     pymongo.ASCENDING)]
)

'num_1'

In [176]:
for index in db.things.list_indexes():
    pprint.pprint(index)

pprint.pprint(db.things.index_information())

SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('num', 1)])), ('name', 'num_1')])
{'_id_': {'key': [('_id', 1)], 'v': 2}, 'num_1': {'key': [('num', 1)], 'v': 2}}


In [177]:
db.things.find({"num": {"$gt":80}}).explain()

{'queryPlanner': {'plannerVersion': 1,
  'namespace': 'documents.things',
  'indexFilterSet': False,
  'parsedQuery': {'num': {'$gt': 80}},
  'winningPlan': {'stage': 'FETCH',
   'inputStage': {'stage': 'IXSCAN',
    'keyPattern': {'num': 1},
    'indexName': 'num_1',
    'isMultiKey': False,
    'multiKeyPaths': {'num': []},
    'isUnique': False,
    'isSparse': False,
    'isPartial': False,
    'indexVersion': 2,
    'direction': 'forward',
    'indexBounds': {'num': ['(80, inf.0]']}}},
  'rejectedPlans': []},
 'executionStats': {'executionSuccess': True,
  'nReturned': 19,
  'executionTimeMillis': 2,
  'totalKeysExamined': 19,
  'totalDocsExamined': 19,
  'executionStages': {'stage': 'FETCH',
   'nReturned': 19,
   'executionTimeMillisEstimate': 0,
   'works': 20,
   'advanced': 19,
   'needTime': 0,
   'needYield': 0,
   'saveState': 0,
   'restoreState': 0,
   'isEOF': 1,
   'docsExamined': 19,
   'alreadyHasObj': 0,
   'inputStage': {'stage': 'IXSCAN',
    'nReturned': 19,
    

In [172]:
db.things.drop_index("num_1")

In [183]:
cursor = db.things.find({}) \
.limit(12) \ # 보여주는 데이터 제한
.skip(1) # 처음에 보여줄 데이터 제한

for item in cursor:
    pprint.pprint(item)

{'_id': ObjectId('5ff2cabf30cd3ec39f73236e'), 'num': 1}
{'_id': ObjectId('5ff2cabf30cd3ec39f73236f'), 'num': 2}
{'_id': ObjectId('5ff2cabf30cd3ec39f732370'), 'num': 3}
{'_id': ObjectId('5ff2cabf30cd3ec39f732371'), 'num': 4}
{'_id': ObjectId('5ff2cabf30cd3ec39f732372'), 'num': 5}
{'_id': ObjectId('5ff2cabf30cd3ec39f732373'), 'num': 6}
{'_id': ObjectId('5ff2cabf30cd3ec39f732374'), 'num': 7}
{'_id': ObjectId('5ff2cabf30cd3ec39f732375'), 'num': 8}
{'_id': ObjectId('5ff2cabf30cd3ec39f732376'), 'num': 9}
{'_id': ObjectId('5ff2cabf30cd3ec39f732377'), 'num': 10}
{'_id': ObjectId('5ff2cabf30cd3ec39f732378'), 'num': 11}
{'_id': ObjectId('5ff2cabf30cd3ec39f732379'), 'num': 12}


In [184]:
db.things.find_one() # 처음 하나만 리턴

{'_id': ObjectId('5ff2cabf30cd3ec39f73236d'), 'num': 0}

In [185]:
# 범위 쿼리 연산자  
# $lt : ~보다 작은  
# $gt : ~보다 큰  
# $lte : ~보다 작거나 같은  
# $gte : ~보다 크거나 같은  

# 집합 연산자
# $in : 어떤 인수든 하나라도 참고 집합에 있는 경우 일치
# $all : 모든 인수가 참고 집합에 있고 배열이 포함된 도큐먼트에서 사용되는 경우 일치
# $nin : 그 어떤 인수도 참고 집합에 있지 않을 경우 일치

In [188]:
# $project : 출력 도큐먼트 상에 배치할 필드를 지정한다
# $match : 처리될 도큐먼트를 선택하는 것. find()와 비슷한 역할을 수행한다
# $limit : 다음 단계에 전달될 도큐먼트의 수를 제한한다.
# $skip : 지정된 수의 도큐먼트를 건너뛴다.
# $unwind : 배열을 확장하여 각 배열 항목에 대해 하나의 출력 도큐먼트를 생성한다.
# $group : 지정된 키로 도큐먼트를 그룹화한다.
# $sort : 도큐먼트를 정렬한다.
# $geoNear : 지리 공간위치 근처의 도큐먼트를 선택한다.
# $out : 파이프라인의 결과를 컬렉션에 쓴다.
# $redact : 특정 데이터에 대한 접근을 제어한다
